In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64
import random
import math
import doctest
import time
import hashlib

import concurrent.futures

In [2]:
def key_generator_AES(kp):
    """
    Génere une clef de 16 octets avec kp bits significatives
    >>> kp = 10
    >>> len(key_generator_AES(kp)) == 16
    True
    """
    prng = random.SystemRandom()
    n = prng.getrandbits(kp)
    n = n.to_bytes(16, byteorder='little')
    return n

In [3]:
def extract_bits(x, kp):
    """         
    Extrait kp bits de x, où x est de type byte
    >>> kp = 10
    >>> x = key_generator_AES(kp)
    >>> len (extract_bits(x, kp) ) == (kp//8 + 1)
    True
    """
    res = list(x[0:kp//8])
    tmp = kp - ((kp//8)*8)
    if  tmp > 0 :
        res.append(x[kp//8] &  ((2**tmp)-1))   
    return bytes(res)

In [4]:
def simple_enc_AES(msg, key, mode = 'MODE_ECB'):
    """
    Chiffrer AES
    Retourne le message chiffré par l'AES en bytes
    """
    if not (isinstance(msg,bytes)):
        msg = msg.encode()
        
    cipher = AES.new(key, AES.MODE_ECB)
    enc = cipher.encrypt(pad(msg, len(key)))
    return enc

def simple_dec_AES(msg, key, mode = 'MODE_ECB'):
    """
    Dechiffrer AES
    Retourne le message non unpader et en bytes
    
    ATTENTION => le retour n'est pas UNPADER, fo utiliser unpad !!!
    
    >>> msg = 'test'
    >>> key = key_generator_AES(10)
    >>> encrypt = simple_enc_AES(msg, key)
    >>> unpad(simple_dec_AES(encrypt,key),16).decode()
    'test'
    """
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(msg)
    
    # error de padding si clef incorrevt
    #return unpad(plaintext, len(key))  
    return plaintext

In [5]:
def double_AES(msg, key1, key2,  mode = 'MODE_ECB'):
    """
    Faire le double chiffrement AES
    en passant les 2 clefs key1 et key2 en parametres
    (on a pas defini les autres modes)
    """
    enc = simple_enc_AES(msg, key1)
    enc = simple_enc_AES(enc, key2)
    return enc

In [6]:
kp = 3*8
key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
M1 = "Voici le message 1"
C1 = double_AES(M1,key1,key2)
M2 = "Voici le message 2"
C2 = double_AES(M2,key1,key2)

In [7]:
def new_step_v1(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> type(new_step_v2(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    >>> len(new_step_v1(simple_enc_AES, 'test', 10, x)) == 16
    True
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = hashlib.blake2b(c).digest()
    c = hashlib.sha1(c).digest()
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp-1):
        c[i] = ((((c[i]+c[i+1])^c[i+1])-c[i+2])&255)^((((~c[i]+c[i+1])^c[i+1])-~c[i+2])&255)
    c[tmp-1] = ((((c[tmp-1]+c[0])^c[0])-c[1])&255)^((((~c[tmp-1]+c[0])^c[0])-~c[1])&255)
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp

In [8]:
def new_step_v2(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> type(new_step_v2(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    >>> len(new_step_v2(simple_enc_AES, 'test', 10, x)) == 16
    True
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00") 
        
    c = f(M, x)  
    c = hashlib.blake2s(c).digest()
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp-1):
        c[i] = (((c[i]^c[i+1])-(~c[i]^~c[i+1]))<<4)&255
    c[tmp-1] = (((c[tmp-1]^c[0])-(~c[tmp-1]^~c[0]))<<4)&255
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)
    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
        
    return tmp

In [9]:
def new_step_v3(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v3(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v3(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp-1):
        c[i] = (((((~c[i] & 255 )^(~c[i+1]))>>4)&255)<<3)&255
    c[tmp-1] = (((((~c[tmp-1] & 255 )^(~c[0]))>>4)&255)<<3)&255
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp

doctest.testmod()

TestResults(failed=0, attempted=18)

In [10]:
def new_step_v4(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v4(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v4(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp-1):
        c[i] = ((((~c[i] & 255 )^(~c[i+1]))>>4)&255)
    c[tmp-1] = ((((~c[tmp-1] & 255 )^(~c[0]))>>4)&255)
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp

In [11]:
def new_step_v5(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v5(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v5(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp):
        c[i] = ((((~c[i] & 255 )^(~c[i+1]))<<4)&255)
    c[tmp-1] = ((((~c[tmp-1] & 255 )^(~c[0]))<<4)&255)
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp

In [12]:
def new_step_v6(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v6(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v6(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp-1):
        t = c[i+1]<<4
        c[i+1] = (c[i]>>2 )&255
        c[i] = (t>>2)&255
    t = c[0]<<4
    c[0] = (c[tmp-1]>>2 )&255
    c[tmp-1] = (t>>2)&255    
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp

In [13]:
def new_step_v7(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v7(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v7(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = bytearray(sorted(c))
    
    tmp = (kp // 8) + 1
    
    for i in range (tmp-1):
        c[i] = c[i+1]^(~c[i]%255)
    c[tmp-1] = c[0]^(~c[tmp-1]%255)   
    
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp

In [14]:
def new_step_v8(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v8(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v8(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = hashlib.sha256(c).digest()
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp

In [15]:
def new_step_v9(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v9(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v9(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp-1):
        c[i] = (((((c[i] & 255)>>4)^(c[i+1]))<<4))&255
    c[tmp-1] = (((((c[tmp-1] & 255)>>4)^(c[0]))<<4))&255
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp


In [16]:
def new_step_v10(f, M, kp, x):
    """
    passer d'un xi au suivant
    
    >>> x = extract_bits(key_generator_AES(13),13)
    >>> len(new_step_v10(simple_enc_AES, 'test', 10, x)) == 16
    True
    >>> type(new_step_v10(simple_enc_AES, 'test', 10, x))
    <class 'bytes'>
    """
    if len(x) < 16:                     # des fois les 8 premiers bits sont nuls
        x = x.rjust(16,b"\x00")       
    c = f(M, x)
    c = hashlib.sha512(c).digest()
    c = bytearray(c)
    tmp = (kp // 8) + 1
    for i in range (tmp-1):
        c[i] = (((((c[i] & 255)>>4)^(c[i+1]))<<4))&255
    c[tmp-1] = (((((c[tmp-1] & 255)>>4)^(c[0]))<<4))&255
    tmp = extract_bits(c,kp)
    tmp += b"\0" * (16 - (kp//8) - 1)    
    if len(tmp) < 16:                     # des fois les 8 premiers bits sont nuls
        tmp = tmp.ljust(16,b"\x00") 
    return tmp


In [17]:
def choice_new_step(version_new_step):
    """
    versio_new_step : un entier 
    Choisir une version de new_step
    """

    if version_new_step == 0:
        return new_step_v1
    if version_new_step == 1:
        return new_step_v2
    if version_new_step == 2:
        return new_step_v3
    if version_new_step == 3:
        return new_step_v4
    if version_new_step == 4:
        return new_step_v5
    if version_new_step == 5:
        return new_step_v6
    if version_new_step == 6:
        return new_step_v7
    if version_new_step == 7:
        return new_step_v8
    if version_new_step == 8:
        return new_step_v9
    if version_new_step == 9:
        return new_step_v10


In [18]:
def trail(f, msg, kp, l, x0, version_new_step):
    """
    Retourne un triplet (x0, xd, d) 
    f : fonction chiffrement OU dechiffrement
    msg : message clair OU chiffré deux fois  
    kp :nb de bit significatif de la clef
    l : nb de bit à 0 (pour la condition d'arrêt)
    
    >>> msg = "Voici le message"
    >>> x0 = key_generator_AES(kp)
    >>> kp = 10
    >>> l = 3
    >>> version_new_step = 2
    >>> (x0,xd,d) = trail(simple_enc_AES, msg, kp, l, x0, version_new_step)
    >>> tmp = x0
    >>> for _ in range(d):
    ...    tmp = new_step_v3(simple_enc_AES, msg, kp, tmp)     
    >>> print(tmp==xd)
    True
    
    >>> mask_l = 2**l - 1 
    >>> xd = extract_bits(xd,kp)
    >>> xd = int.from_bytes(xd, 'big')
    >>> print( xd & mask_l == 0 )
    True
    """
    
    tmp = x0

    d = 0                                # compter le nb de pas
    max_it = (20/ (1/(2**l))) // 3       # diviser par 3 car sinon ca prends tres longtemps
    mask_l = 2**l - 1               
    
    while True:
        
        if len(tmp) < 16:                # des fois les 8 premiers bits sont nuls alors la clef devient trop courte 
            tmp = tmp.rjust(16,b"\x00")  # après la conversion de bits en byte
        if d == max_it:
            #print("Risque de cycle ")
            return None
        
        tmp_binary = extract_bits(tmp,kp) #bytes_to_bin(tmp)[:kp+2]
        tmp_binary = int.from_bytes(tmp_binary, "big")        
        if tmp_binary & mask_l == 0:  # condition d'arret
            xd = tmp
            break
            
        d += 1
        tmp = (choice_new_step(version_new_step))(f, msg, kp, tmp)       
    return (x0, xd, d)


In [19]:
def F(b):
    """
    Choisir une fonction 
    b : 0 OU 1
    
    0 correspond à simple_enc_AES
    1 correspond à simple_dec_AES
    """
    if b == 0:
        return simple_enc_AES
    if b == 1:
        return simple_dec_AES

In [20]:
def remonter (F, A, B, M, C, kp, b, version_new_step):
    """
    Returne ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y)
    F : choix entre chiffrement et déchiffrement 
    A, B : triplet (x0, xd, d)
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    b : 0 ou 1
    """       
    couple = [M,C]
    cpt = 0
    if A[2] >= B[2]: 

        x = A[0]
        for _ in range(A[2]-B[2]):
            x =  (choice_new_step(version_new_step))(F(b), couple[b], kp, x)
        y = B[0]
        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break
            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x =  (choice_new_step(version_new_step))(F(b), couple[b], kp, tmp1)
            y = (choice_new_step(version_new_step))(F(1-b), couple[1-b], kp, tmp2)
            
            if cpt>1000: # boucle ??
                #print("boucle ??")
                return None
            cpt+=1
            
        return (   (tmp1, b) , (tmp2, 1-b)    )
        
        
    else:          # A[2] < B[2] mais on fait la meme chose
        y = B[0]
        for _ in range(B[2]-A[2]):
            y =  (choice_new_step(version_new_step))(F(1-b), couple[1-b], kp, y)
        x = A[0]
        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = (choice_new_step(version_new_step))(F(b), couple[b], kp, tmp1)
            y = (choice_new_step(version_new_step))(F(1-b), couple[1-b], kp, tmp2)
            
            if cpt>1000: # boucle ??
                #print("boucle ?")
                return None
            cpt+=1

        return (   (tmp1, b) , (tmp2, 1-b)    )

In [21]:
def collision_detection(F, M, C, kp, l, dico, version_new_step):
    """
    Detecte une seule collision
    Retourne le couple ( (x, f1) , (y, f2) ) tq x != y et f1(x) == f2(y) 
    F : choix entre chiffrement et déchiffrement 
    M, C : clair et chiffré double
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    dico : contenant les collisions trouvées
    
    >>> kp = 10
    >>> version_new_step = 1
    >>> key1 = key_generator_AES(kp)
    >>> key2 = key_generator_AES(kp)
    >>> M = 'Voici le message'
    >>> C = double_AES(M, key1, key2)
    >>> l = 4
    >>> dico = {}
    >>> res = collision_detection(F, M, C, kp, l, dico, version_new_step)
    >>> if (res[0][1]==0):  
    ...     enc = simple_enc_AES(M,res[0][0])
    ...     dec = simple_dec_AES(C, res[1][0])
    ... elif (res[0][1]==1):
    ...     enc = simple_enc_AES(M,res[1][0])
    ...     dec = simple_dec_AES(C, res[0][0])
    >>> print(enc[:kp//8]==dec[:kp//8] )
    True
    """
    
    couple = [M,C]
    seuil = 10000                  # valeur estimé pour le nb max d'éléments dans le dico
    
    while True: 
        b = random.randint(0,1) 
        x0 = key_generator_AES(kp)           # clef aléatoire initiale
        res = trail(F(b),couple[b], kp, l, x0, version_new_step)
        if res == None:
            continue
            
        x0, xd, d = res

        if (xd,1-b) in dico:        # collision trouvée
            A = (x0,xd,d)                                   # b
            B = (dico[(xd,1-b)][0], xd, dico[(xd,1-b)][1])  # 1-b
            tmp = remonter(F, A , B, M, C, kp, b, version_new_step )
            if(tmp == None):
                continue
            else: 
                return tmp
        
        # si la taille du dico depasse la limite fixée, choix aléatoire de la victime
        if len(dico) >= seuil:      
            delete = random.choice(list(dico.keys()))
            dico.pop(delete)
            
        dico[(xd,b)] = (x0, d)
        

In [22]:
class Statistics:
    n_new_collisions = 0
    n_idem_collisions = 0
    def __init__(self):
        pass

In [43]:
def golden_collision(F, M1, C1, M2, C2 ,kp, l):
    """
    Trouve la golden collision en vérifiant à chaque fois
    les clefs obtenus avec (M2, C2)
    F : choix entre chiffrement et déchiffrement 
    M1, C1 : clair et chiffré double pour trouver les collisions
    M2, C2 : clair et chiffré double pour la vérificaiton
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    """
    dico = {}
    liste = []
    stat = Statistics()
    i=1
    version_new_step = 0
    print("On utilise new_step_v"+str(version_new_step+1))
    
    ancient = stat.n_new_collisions
    while True:
        
        if i%500==0:    
            print("Le nombre de new collisions",stat.n_new_collisions)
            print("Le nombre de collisions idem",stat.n_idem_collisions)
            if ancient == stat.n_new_collisions : # changer de version de new_step  quand l'ancienne version ne trouve plus de nouveau collision                  
                version_new_step = (version_new_step+1) %10
                print("\nOn change de version de NEW_STEP, on utilise maintenant new_step_v"+str(version_new_step+1))
            else :
                ancient = stat.n_new_collisions
                      
        colli = collision_detection(F, M1, C1, kp, l,dico, version_new_step)

        if colli == None:
            print('None')
            continue

        if colli in liste or (colli[1],colli[0]) in liste: # si collision deja trouvé 
            stat.n_idem_collisions += 1
            i+=1    
            continue 
          
        stat.n_new_collisions += 1
        liste.append(colli)
        
        if (len(liste)==((2**kp)*(2**kp))):
            print('GROOS PB !!!!!' )
            return (liste, None)                      # pour debug
        
        
        try: 
            if colli[0][1] == 0:      # 0 correspond a enc
                tmp1 = simple_enc_AES(M2, colli[0][0])
                tmp2 = unpad(simple_dec_AES(C2, colli[1][0]), 16)
            else:
                tmp1 = unpad(simple_dec_AES(C2, colli[0][0]),16)
                tmp2 = simple_enc_AES(M2, colli[1][0])
            if( tmp1 == tmp2):
                
                from IPython.display import clear_output
                clear_output(wait=True)  # clear cell output 
                
                print( "GOLDEN COLLISION ! ")
                print("Voici la collision :",colli)
                print("On a utiliser:",i,"iterations")
                return (liste,colli)              # liste pour debug
        except ValueError:
            pass      
        i+=1

In [141]:
def golden_collision_parr(F, M1, C1, M2, C2 ,kp, l):
    """
    Trouve la golden collision en vérifiant à chaque fois
    les clefs obtenus avec (M2, C2)
    F : choix entre chiffrement et déchiffrement 
    M1, C1 : clair et chiffré double pour trouver les collisions
    M2, C2 : clair et chiffré double pour la vérificaiton
    kp : nb de bits significatif de la clef
    l : nb de bits pour la condition d'arrêt
    """
    dico = {}
    liste = []
    stat = Statistics()
    i=1
    version_new_step = 0
    print("On utilise new_step_v"+str(version_new_step+1))
    ancient = stat.n_new_collisions

    while True:
                     
        #colli = collision_detection(F, M1, C1, kp, l,dico, version_new_step)
        

        pool = concurrent.futures.ProcessPoolExecutor()
        futures = []
        for x in range(500):
            futures.append(pool.submit(collision_detection, F, M1, C1, kp, l,dico, version_new_step))

        #    print(x)
       # print("fin")
        for x in (futures):
           # print("x en cour")
            #sleep(1)
            if i%500==0:  
                print("Le nombre de new collisions",stat.n_new_collisions)
                print("Le nombre de collisions idem",stat.n_idem_collisions)
                if ancient == stat.n_new_collisions : # changer de version de new_step  quand l'ancienne version ne trouve plus de nouveau collision                  
                    version_new_step = (version_new_step+1) %10
                    print()
                    print("On change de version de NEW_STEP, on utilise maintenant new_step_v"+str(version_new_step+1))
                else :
                    ancient = stat.n_new_collisions
                    
            colli = (x.result())
            if colli == None:
                print('None')
                continue

            if colli in liste or (colli[1],colli[0]) in liste: # si collision deja trouvé 
                stat.n_idem_collisions += 1
                i+=1    
                continue 

            stat.n_new_collisions += 1
            liste.append(colli)

            try: 

                if colli[0][1] == 0:      # 0 correspond a enc
                    tmp1 = simple_enc_AES(M2, colli[0][0])
                    tmp2 = unpad(simple_dec_AES(C2, colli[1][0]), 16)
                else:
                    tmp1 = unpad(simple_dec_AES(C2, colli[0][0]),16)
                    tmp2 = simple_enc_AES(M2, colli[1][0])
                if( tmp1 == tmp2):
                    print("AAAAAAAAAAAAAAAAAAAAAAAAAAAA")
                    from IPython.display import clear_output
                    clear_output(wait=True)  # clear cell output 

                    print( "GOLDEN COLLISION ! ")
                    print("Voici la collision :",colli)
                    print("On a utiliser:",i,"iterations")
                    return (liste,colli)              # liste pour debug
            except ValueError:
                pass  
            i+=1

            

In [142]:
kp = 20
l=7

key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
M1 = "Voici le message 1"
C1 = double_AES(M1,key1,key2)
M2 = "Voici le message 2"
C2 = double_AES(M2,key1,key2)

print("Voici la clef 1 : ",key1)
print("Voici la clef 1 : ",key2)

Voici la clef 1 :  b'|Y\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Voici la clef 1 :  b'=o\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [143]:
debut = time.time()
#res = golden_collision(F, M1, C1,M2,C2 ,kp, l )
fin = time.time()
print("Le temps utilisé :", fin-debut)

Le temps utilisé : 7.295608520507812e-05


In [147]:
# verification 
"""
t1 = ((key1,0), (key2,1))
t2 = ((key2,1), (key1,0))
if ((t1) in res[0]) | ((t2) in res[0]) : 
    print("trouvee")
else: 
    print("tres Tres gros PB ")
"""

In [145]:
debut = time.time()
#res = golden_collision_parr(F, M1, C1,M2,C2 ,kp, l )
fin = time.time()
print("Le temps utilisé :", fin-debut)

Le temps utilisé : 4.673004150390625e-05
